# Caracterization of 2-qubit states

We are hoping to caracterize the families of states of the format
$$\rho_{AB}^{\eta} = \eta\rho_{AB}+(1-\eta)\mathbb{I}.$$

I will be creating the target state ($\rho_{AB}$) either by the Bures metric or the Hilbert-Schmidt metric (this part is already finalized).

The tests we are planning on doing is:
- Entanglement
    - PPT test (finding the $\eta_{\text{ent}}$, for which the family begins to be entangled)
        - This part is already finalized.
- Bell's non-locality 
    - Lower bound: SDP
    - Upper bound: Bell's inequalities
- Steering 
    - Upper bound: Sew Saw
    - Lower bound: SDP
        - This part is already finalized.

![Caracterization of the states](caracterization.png)

In [37]:
# Importing the necessary libraries

import numpy as np
from numpy import linalg as LA
from random import random
from scipy.stats import unitary_group

In [38]:
# Functions to create the target state

# Generation of the random matrix from the Ginibre ensemble
def G_matrix(m,n):
    # Matrix G of size m x n
    G = np.zeros((m,n),dtype=np.complex_)
    for k in range(m):
        for l in range(n):
            G[k,l] = random()+random()*1j
    return G

# Generation a random mixed density matrix (Bures metric)
def rho_mixed(n):
    # n = dimension of the state    
    # Create random unitary matrix
    U = unitary_group.rvs(n)
    # Create random Ginibre matrix
    G = G_matrix(n,n)
    # Create identity matrix
    I = np.eye(4)
    # Construct density matrix
    rho = (I+U)*G*(G.conjugate().T)*(I+U.conjugate().T)
    # Normalize density matrix
    rho = rho/(rho.trace())
    return rho

# Generation a random mixed density matrix (Hilbert-Schmidt metric)
def rho_mixed_HS(n):
    # n = dimension of the state  
    # Create random Ginibre matrix
    G = G_matrix(n,n)
    # Construct density matrix
    rho = G*(G.conjugate().T)
    # Normalize density matrix
    rho = rho/(rho.trace())
    return rho

In [39]:
# Generating the target state (Werner state)

#Create the base kets |00>, |01>, |10> and |11>
ket_00 = np.array([[1],[0],[0],[0]])
ket_01 = np.array([[0],[1],[0],[0]])
ket_10 = np.array([[0],[0],[1],[0]])
ket_11 = np.array([[0],[0],[0],[1]])
#Create |psi> = (|01>-|10>)/sqrt(2) and rho = |psi><psi|
psi = (ket_01-ket_10)/np.sqrt(2)
rho_AB = psi@np.transpose(psi)

# Generating the separable state

rho_I = np.eye(4)/4

## Entanglement

The function `Ent_cert(rho)` will say if the state is entangled or not, by calculating the partial transpose of the state and checking if it is positive.

Here we find the $\eta_{\text{ent}}$ in a precision of 


In [40]:
# Defining the function that certifies entanglement

def Ent_cert(rho):
    # Calculate partial transpose
    n = rho.shape
    rho_TA = np.zeros((n[0],n[1]),dtype=np.complex_)
    a = int(n[0]/2)
    b = int(n[1]/2)
    rho_TA[:a,:b] = rho[:a,:b]
    rho_TA[a:,b:] = rho[a:,b:]
    rho_TA[a:,:b] = rho[a:,:b].T
    rho_TA[:a,b:] = rho[:a,b:].T
    # v - eigenvectors, w - eigenvalues
    w, v = LA.eig(rho_TA)
    # PPT Criterion: Are all eigenvalues >=0?
    if all(i >= 0 for i in w):
        # print('Yes: separable state.')
        ppt = 0
    else:
        # print('No: entangled state.')
        ppt = 1
    return w,v,ppt

In [41]:
# Calculating the limit of entanglement

r = 0.5
eps = 0.5
ppa = 0

i = 0

while eps >= 10**(-15) and r>=0 and r<=1 and i<=10**5:
    rho_ent = r*rho_AB+(1-r)*rho_I
    w,v, ppt = Ent_cert(rho_ent)

    eps = eps/2

    if ppt == 0:
        r = r + eps
    else:
        r = r - eps

    i += 1

eta_ent = r

print(eta_ent)

0.33333333333333304


## Lower bound for Bell non-locality
